In [4]:
%matplotlib inline

import os
import time
from pathlib import Path
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pydub import AudioSegment

from torch.utils.data import Dataset, DataLoader
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T
import torch.nn as nn

from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
from torch.optim.lr_scheduler import ReduceLROnPlateau

print(torch.__version__)
print(torchaudio.__version__)

import librosa

from IPython.display import Audio
from torchaudio.utils import download_asset

torch.random.manual_seed(0)

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

TRAINING_DATA_PATH = "C:/Users/Kevin/AppData/LocalLow/kevinjycui/Training"
TESTING_DATA_PATH = "C:/Users/Kevin/AppData/LocalLow/kevinjycui/Testing"

BLENDSHAPE_PARAMS = ["A", "Angry", "Blink", "Blink_L", "Blink_R", "E", "Fun", "I", "Joy", "LookDown", "LookLeft", "LookRight", "LookUp", "Neutral", "O", "Sorrow", "Surprised", "U"]
BONE_PARAMS = ["Chest", "Head", "Hips", "LeftEye", "LeftFoot", "LeftHand", "LeftIndexDistal", "LeftIndexIntermediate", "LeftIndexProximal", "LeftLittleDistal", "LeftLittleIntermediate", "LeftLittleProximal", "LeftLowerArm", "LeftLowerLeg", "LeftMiddleDistal", "LeftMiddleIntermediate", "LeftMiddleProximal", "LeftRingDistal", "LeftRingIntermediate", "LeftRingProximal", "LeftShoulder", "LeftThumbDistal", "LeftThumbIntermediate", "LeftThumbProximal", "LeftToes", "LeftUpperArm", "LeftUpperLeg", "Neck", "RightEye", "RightFoot", "RightHand", "RightIndexDistal", "RightIndexIntermediate", "RightIndexProximal", "RightLittleDistal", "RightLittleIntermediate", "RightLittleProximal", "RightLowerArm", "RightLowerLeg", "RightMiddleDistal", "RightMiddleIntermediate", "RightMiddleProximal", "RightRingDistal", "RightRingIntermediate", "RightRingProximal", "RightShoulder", "RightThumbDistal", "RightThumbIntermediate", "RightThumbProximal", "RightToes", "RightUpperArm", "RightUpperLeg", "Spine", "UpperChest"]

class Bi_RNN(nn.Module):

    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=11, num_layers=2, rnn_type='LSTM'):
        super(Bi_RNN, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers

        #Define the initial linear hidden layer
        self.init_linear = nn.Linear(self.input_dim, self.input_dim)

        # Define the LSTM layer
        self.lstm = eval('nn.' + rnn_type)(self.input_dim, self.hidden_dim, self.num_layers, batch_first=True, bidirectional=True)

        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim * 2, output_dim)

    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, input):
        #Forward pass through initial hidden layer
        linear_input = self.init_linear(input)

        # Forward pass through LSTM layer
        # shape of lstm_out: [batch_size, input_size ,hidden_dim]
        # shape of self.hidden: (a, b), where a and b both
        # have shape (batch_size, num_layers, hidden_dim).
        lstm_out, self.hidden = self.lstm(linear_input)

        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.linear(lstm_out)
        return y_pred

def plot_waveform(waveform, sr, title="Waveform"):
    waveform = waveform.numpy()

    num_channels, num_frames = waveform.shape
    time_axis = torch.arange(0, num_frames) / sr

    figure, axes = plt.subplots(num_channels, 1)
    if num_channels == 1:
        axes.plot(time_axis, waveform[0], linewidth=1)
        axes.grid(True)
    else:
        for axis in axes:
            axis.plot(time_axis, waveform[0], linewidth=1)
            axis.grid(True)
    figure.suptitle(title)

def plot_spectrogram(specgram, title=None, ylabel="freq_bin"):
    fig, axs = plt.subplots(1, 1)
    axs.set_title(title or "Spectrogram (db)")
    axs.set_ylabel(ylabel)
    axs.set_xlabel("frame")
    im = axs.imshow(librosa.power_to_db(specgram), origin="lower", aspect="auto")
    fig.colorbar(im, ax=axs)

class VRMParamsDataset(Dataset):
    """ VRM Parameter Dataset """

    def __init__(self, filename, DATA_PATH=TRAINING_DATA_PATH, transform=None):
        """
        Arguments:
            filename (string): Path to wav/csv files.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.name = Path(filename).stem
        self.DATA_PATH = DATA_PATH
        self.blendshapes_frame = pd.read_csv(DATA_PATH + "/Blendshapes/" + self.name + ".csv")
        self.bones_frame = pd.read_csv(DATA_PATH + "/Bones/" + self.name + ".csv")
        self.transform = transform

    def __len__(self):
        return len(self.blendshapes_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        time_window = self.blendshapes_frame.iloc[idx, 0]

        blendshape_params = self.blendshapes_frame.iloc[idx, 1:]
        blendshape_params = np.asarray(blendshape_params)
        blendshape_params = blendshape_params.astype('float')

        assert time_window == self.bones_frame.iloc[idx, 0]

        bone_params = self.bones_frame.iloc[idx, 1:]
        bone_params = np.asarray(bone_params)
        bone_params = bone_params.astype('float')

        audio_path = self.DATA_PATH + f"/AudioIntervals/{self.name}/{idx}.wav"

        SPEECH_WAVEFORM, SAMPLE_RATE = torchaudio.load(audio_path)
        SPEECH_WAVEFORM = torch.mean(SPEECH_WAVEFORM, dim=0).unsqueeze(0)

        def mfcc_transform(n_fft=2048, win_length=None, hop_length=512, n_mels=256, n_mfcc=256):
            return T.MFCC(
                sample_rate=SAMPLE_RATE,
                n_mfcc=n_mfcc,
                melkwargs={
                    "n_fft": n_fft,
                    "n_mels": n_mels,
                    "hop_length": hop_length,
                    "mel_scale": "htk",
                },
            )

        mfcc = mfcc_transform()(SPEECH_WAVEFORM)

        return mfcc, np.concatenate((blendshape_params, bone_params))

n_epochs = 2
lr = 0.01
n_folds = 5
lstm_input_size = 1
hidden_state_size = 30
batch_size = 30
num_sequence_layers = 2
output_dim = 11
num_time_steps = 4000
rnn_type = 'LSTM'
print_interval = 3000

train = VRMParamsDataset("7-5-2023 12-17-53 AM", TRAINING_DATA_PATH)
test = VRMParamsDataset("7-4-2023 9-54-11 PM", TESTING_DATA_PATH)

train_loader = DataLoader(train, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=False)

model = Bi_RNN(lstm_input_size, hidden_state_size, batch_size, output_dim, num_sequence_layers, rnn_type)
model = model.to(device)

print(train[0])

2.0.1+cpu
2.0.2+cpu


ParameterError: width=187 must be at least 1 and at most (data.shape[-1] - 1) // 2=1

In [2]:
print(train[0])

ParameterError: Audio data must be of type numpy.ndarray

### Training

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_fn = torch.nn.BCEWithLogitsLoss()

for epoch in range(n_epochs):
    for i, (name, label) in enumerate(train):
        pass

### Testing

In [ ]:
num_correct = 0
num_samples = len(test)

model.eval()

with torch.no_grad():
    pass

print(f"Accuracy: {num_correct / num_samples * 100:.4f}%")